In [3]:
!pip install tensorflow

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
import joblib
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('sentimentdataset.csv')

positive_keywords = [
    'Positive', 'Happiness', 'Joy', 'Love', 'Enjoyment', 'Gratitude', 
    'Awe', 'Admiration', 'Amusement', 'Contentment', 'Empowerment', 
    'Compassionate', 'Hope', 'Euphoria', 'Excitement', 'Pride', 
    'Satisfaction', 'Success', 'Enthusiasm', 'Optimism', 'Wonder', 
    'Affection', 'Delight', 'Serenity', 'Tranquility', 'Bliss', 
    'Inspiration', 'Courage', 'Faith', 'Trust', 'Cheer', 
    'Kindness', 'Companionship', 'Support', 'Togetherness', 'Laughter', 
    'Pleasure', 'Radiance', 'Zeal', 'Playfulness', 'Fortitude', 
    'Aspiration', 'Fulfillment', 'Motivation', 'Sincere', 'Uplifting', 
    'Triumph', 'Blessing', 'Savoring', 'Gratifying', 'Radiating', 
    'Vibrant', 'Jubilation', 'Play', 'Sunshine', 'Charming', 
    'Sparkle', 'Dream', 'Affinity', 'Harmony', 'Abundance', 
    'Celebration', 'Encouragement', 'Nurturing', 'Hopefulness', 'Adoration', 
    'Embrace', 'Serendipity', 'Achievement', 'Rejuvenation', 'Wellness', 
    'Joyfulness', 'Festivity', 'Nirvana', 'Elation', 'Lively', 
    'Empathy', 'Satisfaction', 'Rejoice', 'Radiant', 'Cheerfulness', 
    'Glorious', 'Fortunate', 'Heartwarming', 'Magnanimous', 'Benevolence', 
    'Kindheartedness', 'Devotion', 'Passion', 'Playful', 'Resilience', 
    'Tranquil', 'Exuberance', 'Appreciation', 'Nourishing', 'Empowered', 
    'Unity', 'Compassion', 'Invigorate', 'Elated', 'Encouraging', 
    'Awakening', 'Transformative', 'Blissful', 'Fulfilled', 'Positive', 
    'Radiate', 'Sympathy', 'Cherish', 'Amity', 'Daring', 
]

negative_keywords = [
    'Negative', 'Worst', 'Sadness', 'Fear', 'Disgust', 'Anger', 
    'Frustration', 'Boredom', 'Loneliness', 'Grief', 'Despair', 
    'Hate', 'Resentment', 'Jealousy', 'Isolation', 'Heartbreak', 
    'Sorrow', 'Loss', 'Desperation', 'Betrayal', 'EmotionalStorm', 
    'Disappointment', 'Agony', 'Pain', 'Stress', 'Tension', 
    'Gloom', 'Hopelessness', 'Regret', 'Anxiety', 'Nervousness', 
    'Resentful', 'Miserable', 'Discouragement', 'Suffering', 'Unhappiness', 
    'Terror', 'Pessimism', 'Dismay', 'Rage', 'Bitterness', 
    'Misery', 'Desolation', 'Worry', 'Irritation', 'Neglect', 
    'Apathy', 'Lament', 'Sad', 'Woe', 'Suffer', 
    'Displeasure', 'Painful', 'Aggravation', 'Disheartened', 'Cynicism', 
    'Futility', 'Distress', 'Heartache', 'Regrettable', 'Despairing', 
    'Gloominess', 'Weariness', 'Indifference', 'Anguish', 'Perturbation', 
    'Discontent', 'Depressive', 'Devastation', 'Oppression', 'Dread', 
    'Sadness', 'Isolation', 'Dejection', 'Disquiet', 'Disappointment', 
    'Condemnation', 'Conflict', 'Pity', 'Resignation', 'Overwhelm', 
    'Grim', 'Sorrowful', 'Agitated', 'Haunting', 'Harsh', 
    'Torment', 'Dismal', 'Painstaking', 'Negativity', 'Affliction', 
    'Apprehension', 'Despair', 'Unpleasant', 'Frustrating', 'Anguished', 
    'Chagrin', 'Despondent', 'Agonizing', 'Defeat', 'Calamity', 
]

# Hàm để phân loại cảm xúc
def group_sentiment(sentiment):
    sentiment = sentiment.strip()  # Xóa khoảng trắng thừa
    if sentiment in positive_keywords:
        return 'Positive'
    elif sentiment in negative_keywords:
        return 'Negative'
    else:
        return 'Neutral'


# Áp dụng hàm lên cột sentiment
df['Sentiment'] = df['Sentiment'].apply(group_sentiment)

# Tiền xử lý dữ liệu
df['Sentiment'] = df['Sentiment'].str.lower()  # Chuyển về chữ thường
X = df['Text']
y = df['Sentiment']

# Mã hóa nhãn
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Chuyển đổi văn bản thành số
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Mô hình Logistic Regression
log_reg = LogisticRegression()
log_reg.fit(X_train_vectorized, y_train)
y_pred_log_reg = log_reg.predict(X_test_vectorized)
print("Logistic Regression Classification Report:\n", classification_report(y_test, y_pred_log_reg))
joblib.dump(log_reg, 'logistic_regression_model.pkl')
# Lưu vectorizer
joblib.dump(vectorizer, 'vectorizer.pkl')

# Mô hình Random Forest
rf = RandomForestClassifier()
rf.fit(X_train_vectorized, y_train)
y_pred_rf = rf.predict(X_test_vectorized)
print("Random Forest Classification Report:\n", classification_report(y_test, y_pred_rf))
joblib.dump(rf, 'random_forest_model.pkl')

# Mô hình SVM
svm = SVC()
svm.fit(X_train_vectorized, y_train)
y_pred_svm = svm.predict(X_test_vectorized)
print("SVM Classification Report:\n", classification_report(y_test, y_pred_svm))
joblib.dump(svm, 'svm_model.pkl')

# Mô hình LSTM
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Đệm (padding) các chuỗi
X_train_lstm = pad_sequences(X_train_seq, maxlen=100)
X_test_lstm = pad_sequences(X_test_seq, maxlen=100)

# Tạo mô hình LSTM
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=1000, output_dim=128, input_length=100))
lstm_model.add(LSTM(128))
lstm_model.add(Dense(3, activation='softmax'))

# Biên dịch mô hình
lstm_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Huấn luyện LSTM
lstm_model.fit(X_train_lstm, y_train, epochs=5, batch_size=64, validation_data=(X_test_lstm, y_test))

# Lưu tokenizer
joblib.dump(tokenizer, 'tokenizer.pkl')

# Lưu mô hình LSTM
lstm_model.save('lstm_model.keras')


Logistic Regression Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.46      0.58        28
           1       0.71      0.67      0.69        66
           2       0.62      0.79      0.69        53

    accuracy                           0.67       147
   macro avg       0.70      0.64      0.65       147
weighted avg       0.69      0.67      0.67       147

Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.25      0.37        28
           1       0.68      0.64      0.66        66
           2       0.57      0.81      0.67        53

    accuracy                           0.63       147
   macro avg       0.65      0.57      0.57       147
weighted avg       0.64      0.63      0.61       147

SVM Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.11      0.19        28
           1       

In [32]:
import pandas as pd
import joblib
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow import keras
import tensorflow as tf

# Giả sử bạn có một số văn bản mới để dự đoán
new_texts = [
    "I love this product! It's amazing.",
    "This is the worst experience I've ever had.",
    "I'm feeling indifferent about this.",
]

# Tải mô hình Logistic Regression
log_reg_model = joblib.load('logistic_regression_model.pkl')
vectorizer = joblib.load('vectorizer.pkl')  # Nếu bạn đã lưu vectorizer trước đó

# Tiền xử lý cho Logistic Regression
new_texts_vectorized = vectorizer.transform(new_texts)
log_reg_predictions = log_reg_model.predict(new_texts_vectorized)

# Chuyển đổi nhãn dự đoán về định dạng gốc
log_reg_labels = label_encoder.inverse_transform(log_reg_predictions)

# In ra kết quả dự đoán
print("Logistic Regression Predictions:")
for text, label in zip(new_texts, log_reg_labels):
    print(f"Text: '{text}' => Predicted Sentiment: {label}")

# Tương tự cho mô hình Random Forest
rf_model = joblib.load('random_forest_model.pkl')
rf_predictions = rf_model.predict(new_texts_vectorized)
rf_labels = label_encoder.inverse_transform(rf_predictions)

print("\nRandom Forest Predictions:")
for text, label in zip(new_texts, rf_labels):
    print(f"Text: '{text}' => Predicted Sentiment: {label}")

# Tương tự cho mô hình SVM
svm_model = joblib.load('svm_model.pkl')
svm_predictions = svm_model.predict(new_texts_vectorized)
svm_labels = label_encoder.inverse_transform(svm_predictions)

print("\nSVM Predictions:")
for text, label in zip(new_texts, svm_labels):
    print(f"Text: '{text}' => Predicted Sentiment: {label}")

# Dự đoán với LSTM
# Tải mô hình LSTM và tokenizer
lstm_model = keras.models.load_model('lstm_model.keras')
tokenizer = joblib.load('tokenizer.pkl')  # Nếu bạn đã lưu tokenizer trước đó

# Hàm dự đoán sử dụng reduce_retracing
@tf.function(reduce_retracing=True)
def predict_with_lstm(sequences):
    return lstm_model(sequences, training=False)  # Gọi trực tiếp mô hình mà không dùng predict

# Tiền xử lý cho LSTM
sequences = tokenizer.texts_to_sequences(new_texts)
padded_sequences = pad_sequences(sequences, maxlen=100)  # Giả sử độ dài tối đa là 100

# Dự đoán
predictions = predict_with_lstm(tf.convert_to_tensor(padded_sequences))  # Chuyển đổi sang tensor
predicted_classes = np.argmax(predictions.numpy(), axis=1)  # Lấy chỉ số lớp có xác suất cao nhất

# Chuyển đổi chỉ số thành nhãn cảm xúc
sentiment_labels = ['negative', 'neutral', 'positive']  # Thay đổi nếu cần thiết

# Hiển thị kết quả
for text, label in zip(new_texts, predicted_classes):
    print(f"Text: '{text}' => Predicted Sentiment: {sentiment_labels[label]}")


Logistic Regression Predictions:
Text: 'I love this product! It's amazing.' => Predicted Sentiment: positive
Text: 'This is the worst experience I've ever had.' => Predicted Sentiment: positive
Text: 'I'm feeling indifferent about this.' => Predicted Sentiment: negative

Random Forest Predictions:
Text: 'I love this product! It's amazing.' => Predicted Sentiment: positive
Text: 'This is the worst experience I've ever had.' => Predicted Sentiment: positive
Text: 'I'm feeling indifferent about this.' => Predicted Sentiment: positive

SVM Predictions:
Text: 'I love this product! It's amazing.' => Predicted Sentiment: positive
Text: 'This is the worst experience I've ever had.' => Predicted Sentiment: positive
Text: 'I'm feeling indifferent about this.' => Predicted Sentiment: positive
Text: 'I love this product! It's amazing.' => Predicted Sentiment: negative
Text: 'This is the worst experience I've ever had.' => Predicted Sentiment: positive
Text: 'I'm feeling indifferent about this.' =>